In [1]:
import pandas as pd

In [2]:
import numpy as np

In [27]:
from dynamic_portfolio.utils import *
from dynamic_portfolio.preprocess import *

# Loading data and creating clean dataframe

In [ ]:
df = features_creation('META')

In [ ]:
pd.set_option('display.max_columns', None)

# Baseline

In [ ]:
backtest_mask = final_df['date'] > '2020-01-02'

In [ ]:
backtest_df = final_df[backtest_mask]

In [ ]:
model_mask = final_df['date'] <= '2020-01-02'

In [ ]:
model_df = final_df[model_mask]

In [ ]:
model_df[['return']].iloc[-1]

In [ ]:
backtest_df[['return']].iloc[0]

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(backtest_df[['return']].iloc[0],model_df[['return']].iloc[-1])**0.5

In [ ]:
def baseline_model(df):
    df = y_train
    y_pred = y_train[-1]
    return y_pred

In [ ]:
def rmse():
    rmse = (np.mean((y_test[0]-y_pred)**2))**0.5
    return rmse

In [ ]:
X = model_df.drop(columns=['date'])

In [ ]:
y = model_df['return']

In [ ]:
model = model.fit(X,y)

In [ ]:
backtest_df = backtest_df.drop(columns=['date'])

In [ ]:
y_true = backtest_df['return']

In [ ]:
y_true

In [ ]:
backtest_df.iloc[[3]]

In [ ]:
backtest_df

In [ ]:
y_pred = []
backtest_df.reset_index(drop=True, inplace=True)
for i in range(len(backtest_df)):
    pred = model.predict(backtest_df.iloc[[i]])
    y_pred.append(pred)

y_pred = np.array(y_pred)

In [ ]:
y_pred[:,0].shape

In [ ]:
y_true.shape

In [ ]:
rmse = (np.mean((y_true-y_pred[:,0])**2))**0.5

In [ ]:
rmse

# Cross vals

We use sklearn's time series split to break up the data in different folds
We use a sklearn example to run the first tests.

Metrics used : 
 - rmse
 - mae
 - R2

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
ts_cv = TimeSeriesSplit(
    n_splits=20,
    gap=0,
    max_train_size=252,
    test_size=45,
)

In [4]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [5]:
def baseline_model():
    

SyntaxError: incomplete input (671652115.py, line 2)

In [6]:
model = HistGradientBoostingRegressor()

In [ ]:
def evaluate(model, X, y, cv):
    cv_results = cross_validate(
        model,
        X,
        y,
        cv=cv,
        scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error"],
    )
    mae = -cv_results["test_neg_mean_absolute_error"]
    rmse = -cv_results["test_neg_root_mean_squared_error"]
    print(
        f"Mean Absolute Error:     {mae.mean():.6f} +/- {mae.std():.6f}\n"
        f"Root Mean Squared Error: {rmse.mean():.6f} +/- {rmse.std():.6f}"
    )


In [ ]:
import seaborn as sns

In [ ]:
evaluate(model, X, y, ts_cv)

In [ ]:
scores = cross_val_score(model, X, y, cv = ts_cv)

In [ ]:
scores = np.mean(scores)
scores

# PCA Analysis

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components = 0.95)
X_pca = pca.fit_transform(aapl) 
print(pca.n_components_) 

In [ ]:
pca.n_components

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
len(np.unique(pd.DataFrame(pca.components_, columns = aapl.columns).columns))


In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
n_pcs= pca.n_components_ 
# get the index of the most important feature on EACH component
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = aapl.columns
# get the most important feature names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

In [ ]:
n_pcs

In [ ]:
len(np.unique(most_important_names))

In [ ]:
np.unique(most_important_names)

In [7]:
from dynamic_portfolio.cross_validate import *

In [8]:
from sklearn.metrics import mean_squared_error

In [16]:
fold_length = 125 #
fold_stride = 60 #1 trimestre
train_test_ratio = 0.7
input_length = 10
horizon = 1
output_length = 1
stride = 1

In [10]:
def get_folds(
    df: pd.DataFrame,
    fold_length: int,
    fold_stride: int):
    '''
    This function slides through the Time Series dataframe of shape (n_timesteps, n_features) to create folds
    - of equal `fold_length`
    - using `fold_stride` between each fold

    Returns a list of folds, each as a DataFrame
    '''

    folds = []
    for idx in range(0, len(df), fold_stride):
        # Exits the loop as soon as the last fold index would exceed the last index
        if (idx + fold_length) > len(df):
            break
        fold = df.iloc[idx:idx + fold_length, :]
        folds.append(fold)
    return folds

In [11]:
def train_test_split(fold: pd.DataFrame,
                     train_test_ratio: float,
                     input_length: int,
                     horizon: int) :
    '''
    Returns a train dataframe and a test dataframe (fold_train, fold_test)
    from which one can sample (X,y) sequences.
    df_train should contain all the timesteps until round(train_test_ratio * len(fold))
    '''

    # TRAIN SET
    # ======================
    last_train_idx = round(train_test_ratio * len(fold))
    fold_train = fold.iloc[0:last_train_idx, :]

    # TEST SET
    # ======================
    first_test_idx = last_train_idx - input_length
    fold_test = fold.iloc[first_test_idx:, :]

    return (fold_train, fold_test)



In [26]:
def cross_validate_ml(df) :
    '''
    get_folds() create many FOLDS, train_test_split() create a split on ONE FOLDS.
    The goal of this function is to make splits and sequences on each FOLDS.
    Then, apply a model.
    '''
    folds = get_folds(df, fold_length, fold_stride) # 1 - Creating FOLDS
    rmses_model = []
    rmses_baseline = []
    
    for fold_id, fold in enumerate(folds):

        # 2 - CHRONOLOGICAL TRAIN TEST SPLIT of the current FOLD

        (fold_train, fold_test) = train_test_split(fold = fold,
                                                train_test_ratio=train_test_ratio,
                                                input_length = input_length,
                                                horizon=horizon)

        # 3 - Scanninng fold_train and fold_test for SEQUENCES

        X_train, y_train = fold_train, fold_train['return']

        X_test, y_test = fold_test, fold_test['return']

        model = RandomForestRegressor()
        model.fit(X_train, y_train)
        rmse_model = (mean_squared_error(y_test, model.predict(X_test)))**0.5
        rmses_model.append(rmse_model)
        
        #print(type(y_train))
        #rmse_baseline = mean_squared_error(y_test.iloc[[0]], y_train.iloc[[-1]])**0.5
        #rmses_baseline.append(rmse_baseline)
        
    
        
        
        
    return np.mean(rmses_model)
        
        

In [19]:
model, baseline = cross_validate_ml(df)[0], cross_validate_ml(df)[1]

In [ ]:
improvement = baseline/model
improvement

In [ ]:
baseline

In [20]:
model

0.009383829786538582

In [13]:
tickers = return_tickers()

In [24]:
tickers['BF.B']

TypeError: list indices must be integers or slices, not str

In [25]:
from sklearn.ensemble import RandomForestRegressor

In [14]:
len(tickers)

497

In [ ]:
rmses = []


for ticker in tickers:
    df = ready_to_train_df(ticker)
    rmse = cross_validate_ml(df)[0]
    rmses.append(rmse)


/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split.dropna(i

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split.dropna(i

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split.dropna(i

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split.dropna(i

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split.dropna(i

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split.dropna(i

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['return'][0]=0


In [ ]:
np.mean(rmses)